### 5. 
Refer to the Bleichenbacher's attack as mentioned in paper title "Chosen Ciphertext Attacks Against Protocols Based on the RSA Encryption Standard PKCS #1". Follow the underlying steps:


a. Generate RSA key pairs, where the public key encryption exponent “e” should be
3. And consider the message “m” to be your roll number (all small letters). For eg
- "p21cs011".
p and q should be 128 bit prime numbers.


In [59]:
from sympy import isprime
import random
def randomPrimeGen(n):
    p=""
    for i in range(3*n*n):
        p="1"
        # random string of 1s and 0s of length n-1
        p+=format(random.getrandbits(n-1),'0b')
        if(isprime(int(p,2))):
            return p
    return False
def GenModulus(n):
    p=randomPrimeGen(n)
    q=randomPrimeGen(n)
    while(p==q):
        q=randomPrimeGen(n)
    return int(p,2)*int(q,2),int(p,2),int(q,2)



In [60]:
from sympy import gcd
import random
def GenRSA(n):
    N,p,q=GenModulus(n)
    phi=(p-1)*(q-1)

    e=3 #GIVEN
    g = gcd(e, phi)
    while g != 1:
        # e = random.randrange(1, phi)
        
        N,p,q=GenModulus(n)
        phi=(p-1)*(q-1)
        g = gcd(e, phi)
    
    d = pow(e, -1, phi)
    print("phi=",phi)
    return N,e,d

N,e,d=GenRSA(128)
print("The output in base10 is:")
print("N=",N)
print("e=",e)
print("d=",d)

        

phi= 10516111741933424577934926587176972827843524495200932610221056429934113416444
The output in base10 is:
N= 10516111741933424577934926587176972828059331690545005064476200398200880115069
e= 3
d= 7010741161288949718623284391451315218562349663467288406814037619956075610963


b. Encrypt the message “m” using the RSA algorithm with PKCS1.5 padding to
obtain ciphertext “x” (Inbuilt libraries support automatic message conversion
followed by encryption). You can use any cryptographic hashing function for the
same.


In [61]:
import hashlib
import os
#here key_size is taken in BYTES (not bits)(32 bytes=256 bits)
def pkcs1_v1_5_pad(message, key_size):

    hash = hashlib.sha1(message).digest()


    # Create the ASN.1 prefix for SHA-1 add only lenght included here
    asn1 = len(hash)
    asn1=asn1.to_bytes(asn1.bit_length()//8+1, byteorder='big')

    
    # Calculate the number of padding bytes needed
    padding_length = key_size - len(asn1) - len(hash) - 3

  
    padding = b'\xff' * padding_length

 
    padded_message = b'\x00\x02' + padding + b'\x00' + asn1 + hash 
   

    return padded_message

M='b21cs50'
M=M.encode()
print("the padded message is:",pkcs1_v1_5_pad(M,32))

the padded message is: b"\x00\x02\xff\xff\xff\xff\xff\xff\xff\xff\x00\x14|\xc9\xeb;\x9e\x97\xb0\x8e\xe9$\x03\xc6'\xd4\x7f\xb4\x1bR\xe1`"


In [62]:
import base64
def pad_and_encrypt(M,public_key):
    M=M.encode()
    M_bytes=pkcs1_v1_5_pad(M, 32)
    M=int.from_bytes(M_bytes, byteorder='big')

    N,e=public_key
    signed_decimal=pow(M,e,N)



    signed_bytes=signed_decimal.to_bytes((signed_decimal.bit_length() + 7) // 8, 'big')

    signed_base64=base64.b64encode(signed_bytes)
    return signed_base64

M='b21cs050'

encrypted=pad_and_encrypt(M,(N,e))
print("The encrypted message is:")
print(encrypted)



The encrypted message is:
b'A/vn0QXXyh6x0xAPnu4rKS3DLvCi8fNUrO4zzK9gshg='


c. Design a server (or function) that accepts the ciphertext, decrypts it using the
private key and verifies the message (plaintext) has the first two bytes as 00h 01h.
The server returns a boolean value indicating the verification results. For instance
- If the first two bytes are anything other than 00h 01h, the server responds with
“False” value. On the other hand, the server returns “True” value.


In [63]:
def decrypt(signed_base64,private_key):
    signed_bytes=base64.b64decode(signed_base64)
    signed_decimal=int.from_bytes(signed_bytes, 'big')
    
    N,d=private_key
    M=pow(signed_decimal,d,N)

    M_bytes=M.to_bytes((M.bit_length() + 7) // 8, 'big')
    if(len(M_bytes)<32):
        M_bytes=bytes(32-len(M_bytes))+M_bytes
    return M_bytes


def pkcs1_v1_5_verify(padded_message, key_size):
    # Check the initial bytes
    if padded_message[:2] != b'\x00\x02':
        return False

    
    return True


In [64]:
def padding_oracle(Cyphertext):
    
    decrypted=decrypt(Cyphertext,(N,d))[:32]

    veri=pkcs1_v1_5_verify(decrypted, 32)
    return veri

print("The result of the padding oracle on original signed message is:")

result=padding_oracle(encrypted) 
print(result)

The result of the padding oracle on original signed message is:
True


In [65]:
def base64_to_decimal_multiply_bytes(signed_base64,num):
    signed_bytes=base64.b64decode(signed_base64)
    signed_decimal=int.from_bytes(signed_bytes, 'big')
    signed_decimal=((signed_decimal%N)*(num%N))%N
    signed_bytes=signed_decimal.to_bytes((signed_decimal.bit_length() + 7) // 8, 'big')
    signed_base64=base64.b64encode(signed_bytes)
    return signed_base64



Your task is to decrypt the ciphertext “x” using the server defined in this question
following the Bleichenbacher's attack.

In [66]:
# from math import ceil
# # Bleichenbacher's attack
k=32
B=(2**(256-16))
B_bytes=B.to_bytes((B.bit_length() + 7) // 8, 'big')
print("The value of B is:",B)
print("length of B in bytes is:",len(B_bytes))
B_bytes=bytes(32-len(B_bytes))+B_bytes
print("The value of B is:",B_bytes)
print("length of B in bytes is:",len(B_bytes))



s=2;

while padding_oracle(base64_to_decimal_multiply_bytes(encrypted,pow(s,e,N)))==False:
    
    s+=1
print("The value of s is:",s)
print(decrypt(base64_to_decimal_multiply_bytes(encrypted,pow(s,e,N)),(N,d)).hex())





The value of B is: 1766847064778384329583297500742918515827483896875618958121606201292619776
length of B in bytes is: 31
The value of B is: b'\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'
length of B in bytes is: 32


The value of s is: 43648
0002045a02cdc4d05f83b61581d573f659f3ca90093378924e865dd0a0779bc2


In [67]:
import math
def ceil(a,b):
    return math.ceil(a/b)

def floor(a,b):
    return math.floor(a/b)

In [68]:

s1=ceil(N,(3*B))
print ("[-] Starting search for s1 (from value %i)" % s1)

it=1;
while True:
    if padding_oracle(base64_to_decimal_multiply_bytes(encrypted,pow(s1,e,N)))==True:
        break
    s1+=1
    it+=1
print("search for s1 ended after",it,"iterations")
print("The value of s1 is:",s1)

[-] Starting search for s1 (from value 1984)
search for s1 ended after 41665 iterations
The value of s1 is: 43648


In [69]:
B2,B3=2*B,3*B

if(B2>B3):
    print("[-] B2 is greater than B3")
else:
    print("[-] B2 is less than B3")
    
newM=set([])
print(newM)
for r in range(ceil((B2*s1 - B3 + 1),N),floor(((B3-1)*s1 - B2),N) + 1):
 
    aa = ceil(B2 + r*N,s1)                          
    bb = floor(B3 - 1 + r*N, s1)        
    new_a = max(B2,aa)
    new_b = min(B3-1,bb)    
    if(new_a<=new_b):
       
        newM |= set([new_a,new_b])
   

    print("r=",r,"new_a=",new_a,"new_b=",new_b)



print("The set of possible messages is:")
print(newM)

[-] B2 is less than B3
set()
r= 15 new_a= 3614030650273344248446899708829014375416789187975666840602217433555533824 new_b= 3614071129724058525788164807768495391191938911736990912650309621591834624
r= 16 new_a= 3854960629698138261866923112002639974586838816891993814398552043652382720 new_b= 3855001109148852539208188210942120990361988540653317886446644231688683520
r= 17 new_a= 4095890609122933059924663438511361053230566346766622800989317211753545728 new_b= 4095931088573647337265928537450842069005716070527946873037409399789846528
r= 18 new_a= 4336820588547727073344686841684986652400615975682949774785651821850394624 new_b= 4336861067998441350685951940624467668175765699444273846833744009886695424
r= 19 new_a= 4577750567972521871402427168193707731044343505557578761376416989951557632 new_b= 4577791047423236148743692267133188746819493229318902833424509177987858432
r= 20 new_a= 4818680547397315884822450571367333330214393134473905735172751600048406528 new_b= 4818721026848030162163715670306814345